In [1]:
# 문서ID로 직접 인쇄화면 HTML 크롤링
# DB 저장 기능 제외 버전

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pathlib import Path
import time, re, traceback
import pandas as pd
import json

# ─────────────────────────────────────────────────────────
# 설정
# ─────────────────────────────────────────────────────────
BASE = "http://office.anyfive.com/home/"

# ⭐ 크롤링할 문서ID 목록 (여기에 원하는 문서ID 입력)
TARGET_DOC_IDS = [
    "2002390",
    "2008214",
    "2008497",
]

def safe(s: str) -> str:
    return "".join(c for c in (s or "") if c not in '\\/:*?"<>|').strip() or "untitled"

# ─────────────────────────────────────────────────────────
# 공통 유틸
# ─────────────────────────────────────────────────────────
driver = None

def wait_click_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))

def wait_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def click_in_all_frames(css_list=None, xp_list=None, text_contains=None):
    """여러 프레임을 훑으면서 클릭 시도"""
    driver.switch_to.default_content()
    if css_list:
        for css in css_list:
            els = driver.find_elements(By.CSS_SELECTOR, css)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if xp_list:
        for xp in xp_list:
            els = driver.find_elements(By.XPATH, xp)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if text_contains:
        els = driver.find_elements(By.XPATH, "//*[self::a or self::button or self::li or self::span]")
        for el in els:
            if text_contains in (el.text or ""):
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                driver.execute_script("arguments[0].click();", el)
                return True
    frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
    for fr in frames:
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if css_list:
            for css in css_list:
                els = driver.find_elements(By.CSS_SELECTOR, css)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if xp_list:
            for xp in xp_list:
                els = driver.find_elements(By.XPATH, xp)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if text_contains:
            els = driver.find_elements(By.XPATH, "//*[self::a or self::button or self::li or self::span]")
            for el in els:
                if text_contains in (el.text or ""):
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                    driver.execute_script("arguments[0].click();", el)
                    return True
    driver.switch_to.default_content()
    return False

def switch_into_frame_having(selector_css):
    """selector_css 요소가 보이는 프레임으로 전환"""
    driver.switch_to.default_content()
    if driver.find_elements(By.CSS_SELECTOR, selector_css):
        return True
    for fr in driver.find_elements(By.CSS_SELECTOR, "iframe, frame"):
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if driver.find_elements(By.CSS_SELECTOR, selector_css):
            return True
    driver.switch_to.default_content()
    return False

def back_to_list():
    """상세에서 목록으로 복귀"""
    try:
        driver.switch_to.default_content()
        switch_into_frame_having(".tabConts_headerBx, #apprDocContent")
        
        selectors = [
            "button.k-button[onclick*='onClick_findFormBtn']",
            "button[onclick*='onClick_findFormBtn']",
            "button[title='목록']",
        ]
        for css in selectors:
            els = driver.find_elements(By.CSS_SELECTOR, css)
            if els:
                driver.execute_script("arguments[0].click();", els[0])
                time.sleep(1.5)
                return True
        
        driver.back()
        time.sleep(1.5)
        return True
    except:
        return False

# ─────────────────────────────────────────────────────────
# 인쇄 페이지 HTML 추출
# ─────────────────────────────────────────────────────────
def open_print_and_extract(doc_id, row_data, extract_css=False):
    """
    문서ID로 상세 열고 → 인쇄 JS 호출 → CSS + HTML 추출 → 목록 복귀
    
    Args:
        doc_id: 문서 ID
        row_data: 결과 저장할 딕셔너리
        extract_css: CSS를 추출할지 여부 (첫 문서만 True)
    """
    try:
        # 프레임 확인
        driver.switch_to.default_content()
        switch_into_frame_having("header, .tabConts_headerBx, #apprDocContent")
        time.sleep(0.5)
        
        print("[1단계] 상세 화면 열기...")
        # ⭐ 문서ID로 직접 상세 열기
        driver.execute_script(f"managementDocList.clickGridRow({doc_id});")
        time.sleep(2.5)
        
        # 상세 페이지 로드 대기
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#apprLineTable, .apr_title"))
            )
        except:
            print("  ⚠️ 상세 페이지 로드 대기 실패")
        
        # 제목 추출
        try:
            title_el = driver.find_element(By.CSS_SELECTOR, "span.apr_title")
            row_data["제목"] = title_el.text.strip()
        except:
            row_data["제목"] = ""
        
        print(f"[2단계] 인쇄 버튼 클릭...")
        cur = driver.current_window_handle
        before_handles = set(driver.window_handles)
        
        # 인쇄 JS 호출
        printed = False
        try:
            driver.execute_script(f"""
                if (window.approvalAddition && approvalAddition.print && approvalAddition.print.showPrint) {{
                    approvalAddition.print.showPrint({doc_id}, 'MANAGEMENT');
                }} else if (window.print && window.print.showPrint) {{
                    print.showPrint({doc_id}, 'MANAGEMENT');
                }}
            """)
            printed = True
            print("  JS로 인쇄 호출 성공")
        except Exception as e:
            print(f"  JS 인쇄 호출 실패: {e}")
        
        if not printed:
            btns = driver.find_elements(By.CSS_SELECTOR, "#documentPrint, #printBtn, .btn_print")
            if not btns:
                btns = driver.find_elements(By.CSS_SELECTOR, "button.k-button[onclick*='print.showPrint']")
            if not btns:
                btns = driver.find_elements(By.XPATH, "//button[contains(@onclick,'print.showPrint')]")
            if not btns:
                btns = driver.find_elements(By.XPATH, "//*[self::button or self::a or self::span][contains(normalize-space(.),'인쇄')]")
            if btns:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btns[0])
                driver.execute_script("arguments[0].click();", btns[0])
                print("  인쇄 버튼 클릭 완료")
            else:
                raise RuntimeError("인쇄 진입 수단을 찾지 못했습니다.")
        
        print("[3단계] 인쇄창 대기 중...")
        time.sleep(1.0)
        new_handle = None
        try:
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > len(before_handles))
            new_handle = (set(driver.window_handles) - before_handles).pop()
            driver.switch_to.window(new_handle)
            print("  새 인쇄창으로 전환됨")
        except:
            new_handle = cur
            print("  같은 탭에서 인쇄창 열림")
        
        print("[4단계] 인쇄창 내용 로딩 대기...")
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".content, #apprDocContent"))
        )
        
        print("[5단계] CSS + HTML 추출 중...")
        
        if extract_css:
            print("  [5-1] CSS 템플릿 추출 (첫 문서)")
            extraction_result = driver.execute_script("""
              let allCSS = '';
              
              try {
                for (let i = 0; i < document.styleSheets.length; i++) {
                  const sheet = document.styleSheets[i];
                  try {
                    const rules = sheet.cssRules || sheet.rules;
                    if (rules) {
                      allCSS += '\\n/* StyleSheet ' + i + ': ' + (sheet.href || 'inline') + ' */\\n';
                      for (let j = 0; j < rules.length; j++) {
                        allCSS += rules[j].cssText + '\\n';
                      }
                    }
                  } catch (e) {
                    allCSS += '\\n/* StyleSheet ' + i + ' 접근 불가 */\\n';
                  }
                }
              } catch (e) {}
              
              const styleTags = document.querySelectorAll('style');
              styleTags.forEach((style, idx) => {
                if (style.innerHTML) {
                  allCSS += '\\n/* Inline Style ' + idx + ' */\\n' + style.innerHTML + '\\n';
                }
              });
              
              let fullContent = document.querySelector('.content');
              let fullHtml = fullContent ? fullContent.outerHTML : '';
              
              let contentEl = document.querySelector('#apprDocContent #content');
              if (!contentEl) contentEl = document.querySelector('#content');
              if (!contentEl) contentEl = document.querySelector('#apprDocContent');
              let docBody = contentEl ? contentEl.innerHTML : '';
              
              return {
                cssContent: allCSS,
                fullHtml: fullHtml,
                docBody: docBody
              };
            """)
            all_css = extraction_result.get('cssContent', '')
            print(f"  ✅ CSS 추출 완료: {len(all_css)} chars")
        else:
            print("  [5-1] 기존 CSS 템플릿 사용")
            extraction_result = driver.execute_script("""
              let fullContent = document.querySelector('.content');
              let fullHtml = fullContent ? fullContent.outerHTML : '';
              
              let contentEl = document.querySelector('#apprDocContent #content');
              if (!contentEl) contentEl = document.querySelector('#content');
              if (!contentEl) contentEl = document.querySelector('#apprDocContent');
              let docBody = contentEl ? contentEl.innerHTML : '';
              
              return {
                fullHtml: fullHtml,
                docBody: docBody
              };
            """)
            all_css = None
        
        full_html = extraction_result.get('fullHtml', '')
        doc_body = extraction_result.get('docBody', '')
        
        if not doc_body or len(doc_body.strip()) == 0:
            raise RuntimeError("본문 HTML이 비어있습니다.")
        
        print(f"[6단계] 추출 완료:")
        if all_css:
            print(f"   - CSS: {len(all_css)} chars")
        print(f"   - 전체 HTML: {len(full_html)} chars")
        print(f"   - 본문 HTML: {len(doc_body)} chars")
        
        row_data["fullHtml"] = full_html
        row_data["docBody"] = doc_body
        
        print("[7단계] 인쇄창 닫고 목록 복귀...")
        try:
            if new_handle != cur:
                driver.close()
                driver.switch_to.window(cur)
            else:
                driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"  인쇄창 닫기 실패: {e}")
        
        print("[8단계] 목록 화면 복귀...")
        back_to_list()
        
        print(f"✅ 추출 완료: {doc_id} | {row_data.get('제목', '')[:30]}")
        return (all_css, row_data)
        
    except Exception as e:
        print(f"\n❌ 에러: {type(e).__name__}: {str(e)}")
        traceback.print_exc()
        raise

# ─────────────────────────────────────────────────────────
# 메인 실행
# ─────────────────────────────────────────────────────────
def main():
    global driver
    
    print("="*70)
    print("📋 문서ID 기반 인쇄화면 HTML 크롤링 시작")
    print(f"   대상 문서: {len(TARGET_DOC_IDS)}건")
    print(f"   문서ID: {TARGET_DOC_IDS}")
    print("="*70)
    
    # 브라우저 설정
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-popup-blocking")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(service=Service(), options=opts)
    driver.get(BASE)
    time.sleep(1.2)
    
    input("\n🔐 로그인 완료 후 Enter를 눌러주세요... ")
    
    # 전자결재 → 결재 문서관리 메뉴 진입
    print("\n📁 전자결재 메뉴 진입 중...")
    click_in_all_frames(
        css_list=["a[href='/apr/'].left_menu", "a[href='/apr/']"],
        xp_list=["//a[@href='/apr/']"],
        text_contains="전자결재"
    )
    time.sleep(0.8)
    
    click_in_all_frames(
        css_list=["li[onclick*='managementDoc']"],
        xp_list=["//*[@onclick and contains(.,'managementDoc')]"],
        text_contains="결재 문서관리"
    )
    time.sleep(1.5)
    
    # 프레임 전환
    switch_into_frame_having("header, .tabConts_headerBx")
    
    # CSS 템플릿 - 첫 문서에서만 추출
    global_css_template = None
    
    # 결과 저장용
    details = []
    success_count = 0
    error_count = 0
    
    # 각 문서ID에 대해 크롤링
    for idx, doc_id in enumerate(TARGET_DOC_IDS):
        print(f"\n{'='*70}")
        print(f"[{idx+1}/{len(TARGET_DOC_IDS)}] 문서ID: {doc_id} 처리 중...")
        print("="*70)
        
        row_data = {
            "문서ID": doc_id,
            "제목": "",
            "cssContent": "",
            "fullHtml": "",
            "docBody": "",
            "completeHtml": ""
        }
        
        try:
            # CSS는 첫 문서에서만 추출
            extracted_css, result = open_print_and_extract(doc_id, row_data, extract_css=(idx == 0))
            
            # 첫 문서에서 CSS 템플릿 저장
            if idx == 0 and extracted_css:
                global_css_template = extracted_css
                print(f"\n🎨 CSS 템플릿 저장: {len(global_css_template)} chars")
            
            details.append(result)
            success_count += 1
            
        except Exception as e:
            error_count += 1
            print(f"  ❌ 실패: {str(e)}")
            
            try:
                back_to_list()
                time.sleep(1)
            except:
                pass
            
            continue
    
    # ─────────────────────────────────────────────────────────
    # 결과 저장
    # ─────────────────────────────────────────────────────────
    print("\n" + "="*70)
    print("📊 크롤링 완료!")
    print("="*70)
    print(f"  ✅ 성공: {success_count}건")
    print(f"  ❌ 실패: {error_count}건")
    
    if details:
        # CSS 템플릿을 모든 문서에 적용
        if global_css_template:
            print(f"\n🎨 CSS 템플릿 적용 중...")
            for doc in details:
                doc["cssContent"] = global_css_template
                doc["completeHtml"] = f"""<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{doc.get('제목', '전자결재 문서')}</title>
    <style>
{global_css_template}
    </style>
</head>
<body>
{doc.get('fullHtml', '')}
</body>
</html>"""
            print("  ✓ 모든 문서에 CSS 적용 완료")
        
        # DataFrame 생성 및 저장
        df = pd.DataFrame(details)
        
        df.to_excel("전자결재_완전한문서.xlsx", index=False, engine='openpyxl')
        print(f"\n📁 저장 완료:")
        print(f"   - 전자결재_완전한문서.xlsx")
        
        # JSON 저장
        with open("전자결재_완전한문서.json", "w", encoding="utf-8") as f:
            json.dump(details, f, ensure_ascii=False, indent=2)
        print(f"   - 전자결재_완전한문서.json")
        
        # 완전한 HTML 파일 개별 저장
        print("\n📁 완전한 HTML 파일 개별 저장 중...")
        Path("완전한문서_HTML").mkdir(exist_ok=True)
        for i, row in enumerate(details, 1):
            doc_id = row.get('문서ID', f'doc_{i}')
            title = safe(row.get('제목', f'문서_{i}'))
            filename = f"완전한문서_HTML/{doc_id}_{title}.html"
            
            with open(filename, "w", encoding="utf-8") as f:
                f.write(row.get('completeHtml', ''))
            print(f"  [{i}] {filename}")
    
    print("\n" + "="*70)
    print("📋 샘플 데이터:")
    print("="*70)
    for i, row in enumerate(details[:3], 1):
        print(f"\n[{i}] 문서ID: {row['문서ID']}")
        print(f"    제목: {row.get('제목', 'N/A')[:50]}")
        print(f"    CSS: {len(row.get('cssContent', ''))} chars")
        print(f"    전체 HTML: {len(row.get('fullHtml', ''))} chars")
        print(f"    완전한 HTML: {len(row.get('completeHtml', ''))} chars")
    
    print("\n" + "="*70)
    print("✨ 작업 완료!")
    print("="*70)
    
    # driver.quit()

if __name__ == "__main__":
    main()

📋 문서ID 기반 인쇄화면 HTML 크롤링 시작
   대상 문서: 3건
   문서ID: ['2002390', '2008214', '2008497']



🔐 로그인 완료 후 Enter를 눌러주세요...  



📁 전자결재 메뉴 진입 중...

[1/3] 문서ID: 2002390 처리 중...
[1단계] 상세 화면 열기...
[2단계] 인쇄 버튼 클릭...
  JS로 인쇄 호출 성공
[3단계] 인쇄창 대기 중...
  새 인쇄창으로 전환됨
[4단계] 인쇄창 내용 로딩 대기...
[5단계] CSS + HTML 추출 중...
  [5-1] CSS 템플릿 추출 (첫 문서)
  ✅ CSS 추출 완료: 37438 chars
[6단계] 추출 완료:
   - CSS: 37438 chars
   - 전체 HTML: 95932 chars
   - 본문 HTML: 83947 chars
[7단계] 인쇄창 닫고 목록 복귀...
[8단계] 목록 화면 복귀...
✅ 추출 완료: 2002390 | 법인카드 사용내역 정산 보고_5월_편영수K

🎨 CSS 템플릿 저장: 37438 chars

[2/3] 문서ID: 2008214 처리 중...
[1단계] 상세 화면 열기...
[2단계] 인쇄 버튼 클릭...
  JS로 인쇄 호출 성공
[3단계] 인쇄창 대기 중...
  새 인쇄창으로 전환됨
[4단계] 인쇄창 내용 로딩 대기...
[5단계] CSS + HTML 추출 중...
  [5-1] 기존 CSS 템플릿 사용
[6단계] 추출 완료:
   - 전체 HTML: 11128 chars
   - 본문 HTML: 812 chars
[7단계] 인쇄창 닫고 목록 복귀...
[8단계] 목록 화면 복귀...
✅ 추출 완료: 2008214 | 2015년 1월 CARD 사용내역 정산 보고 2015년

[3/3] 문서ID: 2008497 처리 중...
[1단계] 상세 화면 열기...
[2단계] 인쇄 버튼 클릭...
  JS로 인쇄 호출 성공
[3단계] 인쇄창 대기 중...
  새 인쇄창으로 전환됨
[4단계] 인쇄창 내용 로딩 대기...
[5단계] CSS + HTML 추출 중...
  [5-1] 기존 CSS 템플릿 사용
[6단계] 추출 완료:
   - 전체 HTML: 48275 chars
   - 본문 HTML: 3693